<a href="https://colab.research.google.com/github/BizziBerry/jsbasic-20220606_anelia-education/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22se4enov_diet_evaluation_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создание ИИ-системы для автоматизированной оценки рациона на основе данных о покупках продуктов из чеков.

**Объект исследования:** Данные из чеков супермаркетов, содержащих текстовые записи о товарах.
. <br>
**Предмет исследования:** Детектирование и классификация данных из чеков по соответствующим категориям/разделам.<br>
**Цель работы:** Разработать систему, которая с высокой точностью распознает названия товаров в чеках и классифицирует их по соответствующим категориям, используя методы NLP и машинного обучения, с возможностью последующей интеграции с телеграм-ботом для взаимодействия с пользователями.<br>
**Задачи работы:**<br>
1.Изучить предоставленными данные с наименованием продуктов из чеков и справочником разделов,выбрать метод обработки данных.<br>
2.Выбрать метод анализа данных,метрики для оценки производительности и параметры работы сети.<br>
3.Сформировать,промаркировать и описать набор данных,выделить классы,участвующие в исследовании.<br>
4.Выполнить предобработку данных.<br>
5.Обучить нейронную сеть.<br>
6.Протестировать обученную модель.<br>
7.Создать отчет о проделанной работе.

# Формирование набора данных


Общий объем набора данных составляет 100 тысяч наименований продуктов из чеков в файле xls которые необходимо классифицировать на 14 разделов, согласно предоставленному справочнику с разделами и составом продуков:



# Импорт необходимых библиотек,загрузка набора данных


In [ ]:
from google.colab import files
from google.colab import drive
import os
import glob
from IPython.display import clear_output
import tensorflow as tf
from tensorflow.python.client import device_lib
drive.mount('/content/drive')
root = '/drive/My Drive/'

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
import cv2
!pip3 install google-colab
from google.colab.patches import cv2_imshow
clear_output()

In [ ]:
import cv2
!pip3 install google-colab
from google.colab.patches import cv2_imshow

import random
from PIL import Image, ImageDraw
import numpy as np
from numpy import int64
import matplotlib.pyplot as plt
import os
import glob
from imageio import imread
import cv2
from google.colab.patches import cv2_imshow
import imutils
import json

import lightgbm as lgb
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

from IPython.display import display
from scipy.stats import chi2_contingency, f_oneway
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_validate, RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

import torch
from IPython.display import Image
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Загружаем даннные

In [ ]:
# Load the newly uploaded Excel files
cheks_file_path = '/content/drive/MyDrive/Colab Notebooks/se4enov/cheks 100.xlsx'
compound_file_path = '/content/drive/MyDrive/Colab Notebooks/se4enov/compound1.xlsx'

# Read the sheets into dataframes
cheks_df = pd.read_excel(cheks_file_path)
compound_df = pd.read_excel(compound_file_path)

# Display the first few rows of both dataframes to understand their structure
cheks_df_head = cheks_df.head()
compound_df_head = compound_df.head()

cheks_df_head, compound_df_head


(                               Наименование продукта
 0                   Флуконазол капс 150мг №1 Вертекс
 1                                     Молоко 3,2%,шт
 2   Борщ Станичный с тушенкой, 103 ккал (завод, з/у)
 3                          Компот из изюма, 114 ккал
 4  Макаронные изделия отварные (масло сливочное),...,
                                Наименование продукта  \
 0  Большой французский хот-дог с сосиской Альпийс...   
 1  Капучино + Французский Хот-дог с сосиской Фран...   
 2                        0,3 Нап. мол. Кофе Капучино   
 3                                           КАПУЧИНО   
 4                Кофе Капучино - комплект для порции   
 
                        Подкатегория  \
 0         Хот-дог (сосиска в хлебе)   
 1         Хот-дог (сосиска в хлебе)   
 2  Кофе на молоке по 2-154 11.1.8.3   
 3  Кофе на молоке по 2-154 11.1.8.3   
 4  Кофе на молоке по 2-154 11.1.8.3   
 
                                             Раздел  Unnamed: 3 Unnamed: 4  
 0          

Выведем первые десять строк датафреймов для проверки корректности чтения excel-файлов.

In [ ]:
display(cheks_df.head(10))

,Наименование продукта,cleaned_name
0,Флуконазол капс 150мг №1 Вертекс,флуконазол капс 150мг 1 вертекс
1,"Молоко 3,2%,шт",молоко 32шт
2,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",борщ станичный с тушенкой 103 ккал завод зу
3,"Компот из изюма, 114 ккал",компот из изюма 114 ккал
4,"Макаронные изделия отварные (масло сливочное),...",макаронные изделия отварные масло сливочное 24...
5,"Филе Горизонт (филе птицы), 218 ккал",филе горизонт филе птицы 218 ккал
6,3 Бумага Упак 1/Прочи/В асс,3 бумага 1прочив асс
7,4 Бумага Упак 1/Прочи/В асс,4 бумага 1прочив асс
8,Брюки спорт мал SAMO M-1016,брюки спорт мал samo m1016
9,Бюстгальтер БН137,бюстгальтер бн137


In [ ]:
display(compound_df.head(10))


,Наименование продукта,Подкатегория,Раздел,Unnamed: 3,Unnamed: 4,cleaned_name
0,Большой французский хот-дог с сосиской Альпийс...,Хот-дог (сосиска в хлебе),Фастфуд 1,NaN,NaN,большой французский хотдог с сосиской альпийск...
1,Капучино + Французский Хот-дог с сосиской Фран...,Хот-дог (сосиска в хлебе),Фастфуд 1,NaN,NaN,капучино французский хотдог с сосиской франкфу...
2,"0,3 Нап. мол. Кофе Капучино",Кофе на молоке по 2-154 11.1.8.3,"11.1.8 КОФЕ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спирта)",NaN,NaN,03 нап мол кофе капучино
3,КАПУЧИНО,Кофе на молоке по 2-154 11.1.8.3,"11.1.8 КОФЕ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спирта)",NaN,NaN,капучино
4,Кофе Капучино - комплект для порции,Кофе на молоке по 2-154 11.1.8.3,"11.1.8 КОФЕ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спирта)",NaN,NaN,кофе капучино комплект для порции
5,"Кофе Капучино 0,2",Кофе на молоке по 2-154 11.1.8.3,"11.1.8 КОФЕ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спирта)",NaN,NaN,кофе капучино 02
6,"Кофе Капучино 0.2л БиФ, шт",Кофе на молоке по 2-154 11.1.8.3,"11.1.8 КОФЕ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спирта)",NaN,NaN,кофе капучино 02л биф
7,"Кофе Капучино 0.2л, шт",Кофе на молоке по 2-154 11.1.8.3,"11.1.8 КОФЕ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спирта)",NaN,NaN,кофе капучино 02л
8,Кофе Капучино 1 порц,Кофе на молоке по 2-154 11.1.8.3,"11.1.8 КОФЕ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спирта)",NaN,NaN,кофе капучино 1 порц
9,Кофе Капучино 1 порц.,Кофе на молоке по 2-154 11.1.8.3,"11.1.8 КОФЕ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спирта)",NaN,NaN,кофе капучино 1 порц


In [ ]:
display(cheks_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99869 entries, 0 to 99868
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Наименование продукта  99740 non-null  object
dtypes: object(1)
memory usage: 780.4+ KB


None

In [ ]:
display(compound_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82372 entries, 0 to 82371
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Наименование продукта  82372 non-null  object 
 1   Подкатегория           82345 non-null  object 
 2   Раздел                 54339 non-null  object 
 3   Unnamed: 3             0 non-null      float64
 4   Unnamed: 4             3 non-null      object 
dtypes: float64(1), object(4)
memory usage: 3.1+ MB


None

#Очистка данных

Шаг 1: Удаление лишних пробелов

In [ ]:
def remove_extra_spaces(name):
    # Удаление начальных и конечных пробелов и замена множественных пробелов одним
    return re.sub(r'\s+', ' ', name).strip()


Шаг 2: Приведение к единому регистру

In [ ]:
def to_lowercase(name):
    return name.lower()


Шаг 3: Удаление специальных символов

In [ ]:
def remove_special_characters(name):
    return re.sub(r'[^\w\s]', '', name)


Шаг 4: Исправление типичных опечаток и сокращений

In [ ]:
def fix_common_typos(name):
    # Примеры исправлений типичных опечаток и сокращений
    corrections = {
        'молоко0': 'молоко',
        'хлеб ': 'хлеб',
        'яблоки': 'яблоко',
        # Добавьте другие исправления по необходимости
    }

    for typo, correction in corrections.items():
        name = name.replace(typo, correction)

    return name


Шаг 5: Устранение лишних слов

In [ ]:
def remove_unnecessary_words(name):
    # Список лишних слов, которые нужно удалить
    unnecessary_words = ['шт', 'грамм', 'гр', 'мл', 'кг', 'упак', 'пакет', 'бутылка', 'банка']

    # Удаление лишних слов
    name = ' '.join([word for word in name.split() if word not in unnecessary_words])

    return name


Применение всех шагов
Теперь применим все функции по очереди для очистки данных:

In [ ]:
def clean_product_name(name):
    if not isinstance(name, str):  # Проверка, что ввод является строкой
        return name

    # Применение всех шагов очистки по очереди
    name = remove_extra_spaces(name)
    name = to_lowercase(name)
    name = remove_special_characters(name)
    name = fix_common_typos(name)
    name = remove_unnecessary_words(name)

    return name

Применение к данным
Применим функции очистки данных к обеим таблицам:

In [ ]:
# Применение функций очистки к обоим датафреймам
cheks_df['cleaned_name'] = cheks_df['Наименование продукта'].apply(clean_product_name)
compound_df['cleaned_name'] = compound_df['Наименование продукта'].apply(clean_product_name)

# Просмотр первых строк после очистки
cheks_df_cleaned_head = cheks_df[['Наименование продукта', 'cleaned_name']].head()
compound_df_cleaned_head = compound_df[['Наименование продукта', 'cleaned_name']].head()

cheks_df_cleaned_head, compound_df_cleaned_head

(                               Наименование продукта  \
 0                   Флуконазол капс 150мг №1 Вертекс   
 1                                     Молоко 3,2%,шт   
 2   Борщ Станичный с тушенкой, 103 ккал (завод, з/у)   
 3                          Компот из изюма, 114 ккал   
 4  Макаронные изделия отварные (масло сливочное),...   
 
                                         cleaned_name  
 0                    флуконазол капс 150мг 1 вертекс  
 1                                        молоко 32шт  
 2        борщ станичный с тушенкой 103 ккал завод зу  
 3                           компот из изюма 114 ккал  
 4  макаронные изделия отварные масло сливочное 24...  ,
                                Наименование продукта  \
 0  Большой французский хот-дог с сосиской Альпийс...   
 1  Капучино + Французский Хот-дог с сосиской Фран...   
 2                        0,3 Нап. мол. Кофе Капучино   
 3                                           КАПУЧИНО   
 4                Кофе Капучино - 

Теперь я выполню объединение данных по наименованию продукта, чтобы к каждому продукту из таблицы cheks подтянуть соответствующую категорию из таблицы compound.

In [ ]:
# Merge the two dataframes on the "Наименование продукта" column
merged_df = cheks_df.merge(compound_df[['Наименование продукта', 'Подкатегория', 'Раздел']], on='Наименование продукта', how='left')

# Display the first few rows of the merged dataframe
merged_df.head(20)


,Наименование продукта,cleaned_name,Подкатегория,Раздел
0,Флуконазол капс 150мг №1 Вертекс,флуконазол капс 150мг 1 вертекс,несъедобное,NaN
1,"Молоко 3,2%,шт",молоко 32шт,"Молоко пастеризованное 3,2% жирности",NaN
2,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",борщ станичный с тушенкой 103 ккал завод зу,Борщ украинский,3.1.15 МЯСО ПТИЦЫ
3,"Компот из изюма, 114 ккал",компот из изюма 114 ккал,Компот из сухофруктов,"11.1.4 КОМПОТЫ, БЕЗАЛКОГОЛЬНЫЕ (менее 1,5% спи..."
4,"Макаронные изделия отварные (масло сливочное),...",макаронные изделия отварные масло сливочное 24...,Макароны отварные,NaN
5,"Филе Горизонт (филе птицы), 218 ккал",филе горизонт филе птицы 218 ккал,Несъедобное,NaN
6,3 Бумага Упак 1/Прочи/В асс,3 бумага 1прочив асс,несъедобное,NaN
7,4 Бумага Упак 1/Прочи/В асс,4 бумага 1прочив асс,несъедобное,NaN
8,Брюки спорт мал SAMO M-1016,брюки спорт мал samo m1016,несъедобное,NaN
9,Бюстгальтер БН137,бюстгальтер бн137,несъедобное,NaN


Шаги для сохранения объединённого DataFrame

In [ ]:
# Сохранить объединенный DataFrame в новый Excel файл
merged_df.to_excel('/content/drive/MyDrive/Colab Notebooks/se4enov/merged_cheks_compound.xlsx', index=False)

Проверим таблицы на явные дубликаты.

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99898 entries, 0 to 99897
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Наименование продукта  99769 non-null  object
 1   cleaned_name           99769 non-null  object
 2   Подкатегория           81542 non-null  object
 3   Раздел                 54309 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


In [ ]:
merged_df.shape

(99898, 4)

Загрузим данные.

In [ ]:
compound_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/se4enov/compound.xlsx')
cheks_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/se4enov/cheks 100.xlsx')
catalog_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/se4enov/catalog.xlsx')


Выведем первые пять строк датафреймов для проверки корректности чтения excel-файлов.

In [ ]:
df_dict = {
    'compound': compound_df,
    'cheks': cheks_df,
    'catalog': catalog_df,
}

for df_name, df in df_dict.items():
    print('Таблица', df_name)
    display(df.head())
    print()

Таблица compound


,Наименование продукта,Раздел,Вода,Бел,Жир,нжк,Хол,мдс,Кр,Угл,...,Глутаминовая кислота*,Пролин*,Серии*,Тирозин*,Цистин*,Оксипролин*,Алкоголь,Кофеин,Теобромин,Нитраты
0,Молоко пастеризованное нежирное 1.1.1.1,"1.1.1 МОЛОКО, ЖИРНОСТЬ < 1,0%",91.2,3.0,0.05,0.0,0.0,4.9,0.0,4.9,...,513.449899,280.430397,187.626093,185.608608,26.227303,0.0,0.0,0.0,0.0,0.0
1,Молоко топленое нежирное 1.1.1.2,"1.1.1 МОЛОКО, ЖИРНОСТЬ < 1,0%",91.2,3.0,0.05,0.0,0.0,5.0,0.0,5.0,...,513.449899,280.430397,187.626093,185.608608,26.227303,0.0,0.0,0.0,0.0,0.0
2,"Молоко топленое 1,0% жирности 1.1.1.3","1.1.1 МОЛОКО, ЖИРНОСТЬ < 1,0%",90.4,3.0,1.00,0.7,3.0,4.8,0.0,4.8,...,513.449899,280.430397,187.626093,185.608608,26.227303,0.0,0.0,0.0,0.0,0.0
3,Молоко белковое 1.1.1.4,"1.1.1 МОЛОКО, ЖИРНОСТЬ < 1,0%",87.4,4.3,1.00,0.7,3.0,6.4,0.0,6.4,...,735.944855,401.950235,268.930733,266.039005,37.592468,0.0,0.0,0.0,0.0,0.0
4,Пахта «Идеал» пастеризованная 1.1.1.5,"1.1.1 МОЛОКО, ЖИРНОСТЬ < 1,0%",90.2,3.3,1.00,0.7,3.0,4.7,0.0,4.7,...,564.794889,308.473436,206.388702,204.169469,28.850034,0.0,0.0,0.0,0.0,0.0



Таблица cheks


,Наименование продукта
0,Флуконазол капс 150мг №1 Вертекс
1,"Молоко 3,2%,шт"
2,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)"
3,"Компот из изюма, 114 ккал"
4,"Макаронные изделия отварные (масло сливочное),..."



Таблица catalog


,Наименование вещества,Вода,Бел,Жир,нжк,Хол,мдс,Кр,Угл,пв,...,Глутаминовая кислота*,Пролин*,Серии*,Тирозин*,Цистин*,Оксипролин*,Алкоголь,Кофеин,Теобромин,Нитраты
0,ЕИ,гр,гр,гр,гр,мг,гр,гр,гр,гр,...,мг,мг,мг,мг,мг,мг,гр,мг,мг,мг


Данные загружены корректно. Названия столбцов необходимо привести к единому стилю (преобладающий стиль названий - CamelCase). Целевого столбца в явном виде нет, но наличие или отсутствие даты в столбце 'EndDate' таблицы contract явно говорит о том, ушёл ли клиент или нет, поэтому в дальнейшем необходимо будет выделить из этого столбца целевой признак.

Проверим исходные данные на отсутствие пропусков и соответствие типов данных их значениям.

In [ ]:
for df_name, df in df_dict.items():
    print('Таблица', df_name)
    print(df.info())
    print()

Таблица compound
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329 entries, 0 to 1328
Data columns (total 86 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Наименование продукта   1329 non-null   object 
 1   Раздел                  1329 non-null   object 
 2   Вода                    1329 non-null   object 
 3   Бел                     1329 non-null   float64
 4   Жир                     1329 non-null   float64
 5   нжк                     1329 non-null   float64
 6   Хол                     1329 non-null   float64
 7   мдс                     1329 non-null   float64
 8   Кр                      1329 non-null   float64
 9   Угл                     1329 non-null   float64
 10  пв                      1329 non-null   float64
 11  OK                      1329 non-null   float64
 12  Зола                    1329 non-null   float64
 13  Na                      1329 non-null   int64  
 14  К                      

Проверим таблицы на явные дубликаты.

In [ ]:
for df_name, df in df_dict.items():
    print('В таблице', df_name, df.duplicated().sum(),'дубликатов')

В таблице compound 0 дубликатов
В таблице cheks 141 дубликатов
В таблице catalog 0 дубликатов


In [ ]:
for df_name, df in df_dict.items():
    print('Таблица', df_name, df.shape)


Таблица compound (1329, 86)
Таблица cheks (99869, 1)
Таблица catalog (1, 85)


Выведем дубликаты

Удалим дубликаты

In [ ]:
df_cheks = cheks_df.drop_duplicates()
df_cheks.head()

,Наименование продукта
0,Флуконазол капс 150мг №1 Вертекс
1,"Молоко 3,2%,шт"
2,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)"
3,"Компот из изюма, 114 ккал"
4,"Макаронные изделия отварные (масло сливочное),..."


In [ ]:
df_cheks.shape

(99728, 1)

Вывод

Исходные данные представляют собой четыре таблицы:
таблица contract состоит из 7043 строк и 8 столбцов
таблица personal состоит из 7043 строк и 5 столбцов
таблица internet состоит из 5517 строк и 8 столбцов
таблица phone состоит из 6361 строк и 2 столбцов
Названия столбцов необходимо привести к единому стилю CamelCase.
Целевой столбец находится в неявном виде в столбце 'EndDate' таблицы contract.
Есть неявные пропуски в столбце 'TotalCharges'.
Дубликатов нет.

# Загрузка репозитория

Для модели анализа данн

# Предобработка набора данных

Предобработка данных - неотъемлемый этап в анализе данных и построении моделей машинного обучения. Её цель заключается в тщательной подготовке данных для обеспечения точности, стабильности и эффективности последующего анализа и использования в моделях. Одной из ключевых задач предобработки является обнаружение и исправление ошибок в данных, таких как опечатки или неточности, что позволяет повысить качество результатов.

# Обучение модели<br>


# Тестирование модели  